# Script to Process Geospatial data to the Police Beat

Gabe Morrison

In [1]:
#library imports: 
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
#The below function reads in the spatial data and sets a CRS. CRS (coordinate reference systems) determine 
# how spatial 3D data (because globe is round) get converted into a 2D plane. 
#The CRS I picked is ESPG 3435 which is targetted for Eastern Illinois.  

def read_spatial(path):
    '''
    Function to read spatial data and converts to ESPG 3435
    Input: path to the data file
    Output: a gpd object 
    '''
    file=gpd.read_file(path)
    file=file.to_crs("EPSG:3435")
    return file

In [3]:
beats = read_spatial("../raw_data/Boundaries - Police Beats (current).geojson")
l = read_spatial("../raw_data/CTA_RailStations")
bus = read_spatial("../raw_data/CTA_BusStops")
metra = read_spatial("../raw_data/Metra_Stations")
streets = read_spatial("../raw_data/Street Center Lines.geojson")

In [4]:
#Function to add point data to the beats polygon dataset
#Specifically, this function first performs a "spatial join" each point in the "other dataset" (ex bus station)
#gets the name of the police beat it is in in a new column ("beat_num")
#Then I group by and count the number of points in each beat and join that to the beat dataset


def join_to_beats(polygon_data, poly_unique_id, other_file, new_column_name):
    '''
    A function that creates a new beats gpd with an updated column 
        to count the number of points from the other_file 
    Inputs:
        polygon_data (gpd): the beats gpd (as a polygon)
        poly_unique_id (string): the unique identifier of the polygon spatial data
        other_file (gpd): another point-based gpd. The number of points in each beat will 
                    be added to the beats as a new column
        new_column_name (string): the name of the column to be added to the beats dataframe
    Output:
        the polygon_data dataframe updated with the new column
    Note: .size suggestion from here: https://stackoverflow.com/questions/19384532/get-statistics-for-each-group-such-as-count-mean-etc-using-pandas-groupby
    '''
    spatial_join = gpd.sjoin(other_file, 
                             polygon_data[[poly_unique_id, "geometry"]], 
                             how="inner", 
                             op='intersects')
    count = spatial_join.groupby(poly_unique_id).size().reset_index(name=new_column_name)
    updated_polygon_data = polygon_data.merge(count, on=poly_unique_id, how="left")
    updated_polygon_data[new_column_name].fillna(0, inplace=True)
    return updated_polygon_data

In [5]:
#Run above function to add point data to the beats
beats = join_to_beats(beats, "beat_num", l, "count_l_stops")
beats = join_to_beats(beats, "beat_num", bus, "count_bus_stops")
beats = join_to_beats(beats, "beat_num", metra, "count_metra_stops")

In [6]:
#Filter Out non-pedestrian Streets (ie highways):
#Syntax help: https://www.kite.com/python/answers/how-to-filter-a-pandas-dataframe-with-a-list-by-%60in%60-or-%60not-in%60-in-python
streets_do_not_want = ["ER", "EXPY", "HWY", "PKWY", "ORD", "XR", "TOLL"]
street_filter = ~streets.street_typ.isin(streets_do_not_want)
streets = streets[street_filter]

In [9]:
#Spatial operation to get all strings into exactly one polygon
#Help on overlay code here::https://stackoverflow.com/questions/60794994/geopandas-split-lines-at-intersection-with-polygons-retain-polygon-id-in-new
#Help with renaming after groupby: https://stackoverflow.com/questions/44416287/renaming-columns-after-group-by-and-sum-in-pandas-dataframe

#First: separate line segments so that they fall into exactly one police beat
#Second: same spatial join as above
#Third: Compute distance (in feet) of each segment
#Fourth/Fifth: Keep only the beat_num and distance columns
#Sixth:group by beat and sum all distances in the beat
#seventh:Add the column with the summarized distances back to the beat dataset


streets_separated = gpd.overlay(streets, beats, how="union", keep_geom_type=False).explode().reset_index(drop=True)
streets_separated = gpd.sjoin(streets_separated, 
                             beats[["beat_num", "geometry"]], 
                             how="inner", 
                             op='intersects')
streets_separated["distance"]= streets_separated["geometry"].length
streets_separated = streets_separated[["beat_num_left", "distance"]]
streets_separated.rename(columns={"beat_num_left":"beat_num"}, inplace=True)
streets_separated = streets_separated.groupby("beat_num").sum("distance").reset_index().rename(columns={'distance':'road_distance_ft'})
beats = beats.merge(streets_separated, how ="left", on="beat_num")

In [ ]:
beats